# Python notebook for pre-processing apical image stack.
# --- UNDER DEVELOPMENT ---
Assumes folder directory structure:
<pre><code>  IMAGING
    image_stacks
    notebooks
    results
</code></pre>
Execute the code sequentially, one block at a time, using &lt;shift-return&gt;.

In [1]:
%matplotlib widget

import numpy as np
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
import matplotlib as mpl

from skimage import data, color
from skimage.morphology import binary_erosion, binary_dilation
from skimage.morphology import remove_small_objects

from skimage.transform import hough_circle, hough_circle_peaks, resize
from skimage.feature import canny
from skimage.draw import circle, circle_perimeter
from skimage.restoration import denoise_bilateral, denoise_wavelet
from skimage import filters
from skimage import io
from skimage import exposure

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from ipyfilechooser import FileChooser


In [2]:
#image_stack = "../image_stacks/Mistgcamp-3_0003.tif"
#image_stack = "../image_stacks/MOVEMENT.tif"
image_stack = "../image_stacks/Movement2.tif"
##image_stack = "../image_stacks/modestmovement.tif"
#image_stack = "../image_stacks/lessmovement3.tif"
image_bits = 10


In [3]:
# look up table for image plot color
#   e.g. coolwarm, jet, plasma, gray
lut = mpl.cm.gray


In [4]:
# Load picture
images = io.imread(image_stack)
images = np.float32(images/(2.0**image_bits))
for i in images:
  for l in range(i.shape[0] - 1): # moving average over every two lines
    i[l] = (i[l] + i[l+1]) / 2.0

fig=plt.figure(figsize=(8,8))
plt.imshow(images[0], norm=None, cmap=lut.name)
print(images.shape)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(450, 512, 512)


In [6]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

@interact(
  gn=widgets.FloatSlider(description='image gain',min=1.0, max=5.0, step=0.1, value=1.0),
  sr=widgets.IntRangeSlider(description='stack range',min=0, max=300, step=1, value=[0,5]), 
  bs=widgets.FloatSlider(description='BILATERAL sigma',min=0.0, max=4.0, step=0.1, value=1.0), 
  cs=widgets.FloatSlider(description='CANNY sigma',min=1.0, max=4.0, step=0.1, value=1.8), 
  ct=widgets.IntRangeSlider(description='threshold',min=0, max=100, step=1, value=[9,22]),
  hr=widgets.IntRangeSlider(description='HOUGH radii',min=3, max=25, step=1, value=[10,16]),
  hd=widgets.IntSlider(description='distance',min=5, max=50, step=1, value=10),
  hp=widgets.IntSlider(description='peaks',min=50, max=500, step=10, value=270),
  ht=widgets.FloatSlider(description='threshold',min=0.0, max=1.0, step=0.01, value=0.12),
  cr=widgets.FloatSlider(description='circle ratio',min=1.0, max=2.0, step=0.01, value=1.2))

def f(gn, sr, bs, cs, ct, hr, hd, hp, ht, cr):
  A00 = gn*np.mean(images[sr[0]:sr[1]], axis=0) # the static images
  A0 = A00 / np.amax(A00) # normalize

  fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8,8))
  imageA = color.gray2rgb(np.uint8(255.0*A0))
    
  # identify nuclei (circles)   
  #A = filters.gaussian(A0, sigma=gs) # noise filter
  #A = denoise_wavelet(A0, wavelet_levels=7, multichannel=False, rescale_sigma=False)
  A = denoise_bilateral(A0, sigma_spatial=bs)
  Auint8 = np.uint8(255.0*A)
  edges = canny(Auint8, sigma=cs, low_threshold=ct[0], high_threshold=ct[1])
  hough_radii = np.arange(hr[0], hr[1], 1) # the range of radii to use in search
  hough_res = hough_circle(edges, hough_radii) # look for circles
  accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, min_xdistance=hd, 
                                           min_ydistance=hd, total_num_peaks=hp, 
                                           threshold=ht, normalize=False)

  # remove false positives (bright disks with dark perimeter)
  pix = [] # as an empty list (for the remaining center pixels)
  for center_y, center_x, radius in zip(cy, cx, radii):
    c = circle(center_y, center_x, radius, shape=A0.shape) # central disk
    cp = circle_perimeter(center_y, center_x, radius+1, shape=A0.shape) # perimeter ring
    if (np.mean(imageA[cp]) / np.mean(imageA[c])) > cr:
      pix.append((center_x, center_y)) # dark disks with bright perimeter are OK

  # remove duplicates (close center pixels)
  pix = np.array(pix) # as a numpy array
  tree = cKDTree(pix) # for pairwise distance query
  rows_to_fuse = list(tree.query_pairs(r=8.0))
  p = np.ones(pix.shape[0])           # array of "keep" flags
  p[np.array(rows_to_fuse)[:,0]] = 0  # flag the first of all duplicate pairs for deletion
  pixx = pix[p.astype(bool)]          # the remaining center pixels 

  # draw nuclei centre pixels
  for i in pixx:
    #imageA[i[1], i[0]] = (255,0,0)
    imageA[circle(i[1], i[0], 1.1, shape=A0.shape)] = (255,0,0)
  
  ax.imshow(imageA, norm=None)
  plt.show()    
  return(str(pixx.shape[0]) + " nuclei identified")


interactive(children=(FloatSlider(value=1.0, description='image gain', max=5.0, min=1.0), IntRangeSlider(value…

In [7]:
bs = 1.0 
cs = 1.8
ct = [9,22]
hr = [10,16]
hd = 10
hp = 270
ht = 0.12
cr = 1.2

out = np.zeros((images.shape[0], 512, 512, 3), dtype=np.uint8)
pixx = []

for i in range(3,images.shape[0]-3):
#for i in range(3,10):
  A00 = np.mean(images[i-3:i+4], axis=0)
  A0 = A00 / np.amax(A00) # normalize
  imageA = color.gray2rgb(np.uint8(255.0*A0))
  imageB = color.gray2rgb(np.uint8(255.0*A00))

  # identify nuclei (circles)   
  #A = filters.gaussian(A0, sigma=gs) # noise filter
  #A = denoise_wavelet(A0, wavelet_levels=7, multichannel=False, rescale_sigma=False)
  A = denoise_bilateral(A0, sigma_spatial=bs)
  Auint8 = np.uint8(255.0*A)
  edges = canny(Auint8, sigma=cs, low_threshold=ct[0], high_threshold=ct[1])
  hough_radii = np.arange(hr[0], hr[1], 1) # the range of radii to use in search
  hough_res = hough_circle(edges, hough_radii) # look for circles
  accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, min_xdistance=hd, 
                                           min_ydistance=hd, total_num_peaks=hp, 
                                           threshold=ht, normalize=False)

  # remove false positives (bright disks with dark perimeter)
  pix = [] # as an empty list (for the remaining center pixels)
  for center_y, center_x, radius in zip(cy, cx, radii):
    c = circle(center_y, center_x, radius, shape=A0.shape) # central disk
    cp = circle_perimeter(center_y, center_x, radius+1, shape=A0.shape) # perimeter ring
    if (np.mean(imageA[cp]) / np.mean(imageA[c])) > cr:
      pix.append((center_x, center_y)) # dark disks with bright perimeter are OK

  # remove duplicates (close center pixels)
  pix = np.array(pix) # as a numpy array
  tree = cKDTree(pix) # for pairwise distance query
  rows_to_fuse = list(tree.query_pairs(r=8.0))
  p = np.ones(pix.shape[0])           # array of "keep" flags
  p[np.array(rows_to_fuse)[:,0]] = 0  # flag the first of all duplicate pairs for deletion

  temp = np.full((np.count_nonzero(p),1),np.float(i))
  pp = pix[p.astype(bool)].astype(float)
  pp = np.concatenate((pp,temp),axis=1)
  pp = list(map(tuple,pp)) # the remaining center pixels 

  # draw nuclei centre pixels
  for v in pp:
    imageB[circle(v[1], v[0], 1.1, shape=A0.shape)] = (255,0,0)
  out[i] = imageB

  print(i, "- " + str(np.count_nonzero(p)) + " nuclei identified")
  pixx += pp 

io.imsave("out.tif", out)

3 - 27 nuclei identified
4 - 28 nuclei identified
5 - 29 nuclei identified
6 - 35 nuclei identified
7 - 24 nuclei identified
8 - 24 nuclei identified
9 - 26 nuclei identified
10 - 21 nuclei identified
11 - 24 nuclei identified
12 - 28 nuclei identified
13 - 23 nuclei identified
14 - 26 nuclei identified
15 - 25 nuclei identified
16 - 29 nuclei identified
17 - 29 nuclei identified
18 - 25 nuclei identified
19 - 27 nuclei identified
20 - 24 nuclei identified
21 - 25 nuclei identified
22 - 25 nuclei identified
23 - 22 nuclei identified
24 - 24 nuclei identified
25 - 25 nuclei identified
26 - 22 nuclei identified
27 - 23 nuclei identified
28 - 28 nuclei identified
29 - 24 nuclei identified
30 - 25 nuclei identified
31 - 27 nuclei identified
32 - 28 nuclei identified
33 - 26 nuclei identified
34 - 26 nuclei identified
35 - 26 nuclei identified
36 - 26 nuclei identified
37 - 24 nuclei identified
38 - 22 nuclei identified
39 - 28 nuclei identified
40 - 32 nuclei identified
41 - 29 nuclei iden

In [8]:
import csv

# save responses to CSV file
t = []
with open ("pixx.csv", 'w') as file:
  writer = csv.writer(file)
  for v in pixx:
    writer.writerow('{:3.2e}'.format(x) for x in v)


In [42]:
from mpl_toolkits.mplot3d import Axes3D

plt.close() # frees up memory
fig = plt.figure()
ax = Axes3D(fig)

tp = np.array(pixx)
ax.scatter(tp[:,0],tp[:,1],tp[:,2])

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
from sklearn.cluster import DBSCAN
from mpl_toolkits.mplot3d import Axes3D

plt.close() # frees up memory
fig = plt.figure()
ax = Axes3D(fig)

tp = np.array(pixx)
tpp = tp * [1.0,1.0,0.5]
db = DBSCAN(eps=10, min_samples=10).fit(tpp)
labels = db.labels_
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    #if k == -1:
        # Black used for noise.
        #col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = tp[class_member_mask & core_samples_mask]
    ax.scatter(xy[:, 0], xy[:, 1], xy[:, 2], color=tuple(col))

    #xy = tp[class_member_mask & ~core_samples_mask]
    #ax.scatter(xy[:, 0], xy[:, 1], xy[:, 2], color=tuple(col))

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

print("Clusters in first slice:", np.count_nonzero((tp[labels!=-1]) == 3, axis=0)[2])




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Estimated number of clusters: 121
Estimated number of noise points: 1007
Clusters in first slice: 25


In [44]:
plt.close() # frees up memory
fig = plt.figure()
ax = Axes3D(fig)
ax.set_xlim3d(0,512)
ax.set_ylim3d(0,512)

tpp = tp[labels==5]
#print(tpp)
ax.plot(tpp[:,0],tpp[:,1],tpp[:,2])

#print(tpp)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
from scipy.interpolate import splprep, splev

x = tpp[:,0]
y = tpp[:,1]
z = tpp[:,2]

tckp,u = splprep([x,y,z],s=2000,k=3,nest=-1)
xnew,ynew,znew = splev(np.linspace(0,1,300),tckp)

plt.close() # frees up memory
fig = plt.figure()
ax = Axes3D(fig)
ax.set_xlim3d(0,512)
ax.set_ylim3d(0,512)

ax.plot(xnew,ynew,znew)
#ax.scatter(x,y,z,c='r')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
from numpy import arange, cos, linspace, pi, sin, random
from scipy.interpolate import splprep, splev

# make ascending spiral in 3-space
t=linspace(0,1.75*2*pi,100)

x = sin(t)
y = cos(t)
z = t

# add noise
x+= random.normal(scale=0.1, size=x.shape)
y+= random.normal(scale=0.1, size=y.shape)
z+= random.normal(scale=0.1, size=z.shape)

# spline parameters
s=3.0 # smoothness parameter
k=2 # spline order
nest=-1 # estimate of number of knots needed (-1 = maximal)

# find the knot points
tckp,u = splprep([x,y,z],s=s,k=k,nest=-1)

# evaluate spline, including interpolated points
xnew,ynew,znew = splev(linspace(0,1,400),tckp)

import pylab
pylab.subplot(2,2,1)
data,=pylab.plot(x,y,'bo-',label='data')
fit,=pylab.plot(xnew,ynew,'r-',label='fit')
pylab.legend()
pylab.xlabel('x')
pylab.ylabel('y')

pylab.subplot(2,2,2)
data,=pylab.plot(x,z,'bo-',label='data')
fit,=pylab.plot(xnew,znew,'r-',label='fit')
pylab.legend()
pylab.xlabel('x')
pylab.ylabel('z')

pylab.subplot(2,2,3)
data,=pylab.plot(y,z,'bo-',label='data')
fit,=pylab.plot(ynew,znew,'r-',label='fit')
pylab.legend()
pylab.xlabel('y')
pylab.ylabel('z')

pylab.savefig('splprep_demo.png')

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler


# #############################################################################
# Generate sample data
centers = [[1, 1], [-1, -1], [1, -1]]
X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
                            random_state=0)

X = StandardScaler().fit_transform(X)

# #############################################################################
# Compute DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [ ]:

# https://github.com/Borda/BIRL/blob/master/bm_experiments/bm_comp_perform.py

import matplotlib.pyplot as plt
import numpy as np
import time
import os

from skimage import data, io
from skimage.transform import resize, warp, AffineTransform
from skimage.color import rgb2gray
from skimage.measure import ransac
from skimage.util import random_noise
from skimage.restoration import denoise_bilateral, denoise_wavelet
from skimage.feature import ORB, match_descriptors

def register_image_pair(idx, path_img_target, path_img_source, path_out):
    """ register two images together

    :param int idx: empty parameter for using the function in parallel
    :param str path_img_target: path to the target image
    :param str path_img_source: path to the source image
    :param str path_out: path for exporting the output
    :return tuple(str,float):
    """
    start = time.time()
    # load and denoise reference image
    img_target = 10.0*io.imread(path_img_target)[0]
    img_target = denoise_wavelet(img_target, wavelet_levels=7, multichannel=False, rescale_sigma=False)

    # load and denoise moving image
    img_source = 10.0*io.imread(path_img_source)[50]
    img_source = denoise_bilateral(img_source, sigma_color=0.05,
                                   sigma_spatial=2, multichannel=False)

    # detect ORB features on both images
    detector_target = ORB(n_keypoints=150)
    detector_source = ORB(n_keypoints=150)
    detector_target.detect_and_extract(img_target)
    detector_source.detect_and_extract(img_source)
    matches = match_descriptors(detector_target.descriptors,
                                detector_source.descriptors)
    print(matches)
    
    # robustly estimate affine transform model with RANSAC
    model, inliers = ransac((detector_target.keypoints[matches[:, 0]],
                       detector_source.keypoints[matches[:, 1]]),
                      AffineTransform, min_samples=25, max_trials=500,
                      residual_threshold=0.95)  # 0.95
    print(inliers)
    
    # warping source image with estimated transformations
    img_warped = warp(img_target, model.inverse, output_shape=img_target.shape[:2])
    path_img_warped = os.path.join(path_out, "result.tif")
    io.imsave(path_img_warped, img_warped)

    # summarise experiment
    execution_time = time.time() - start
    return path_img_warped, execution_time 



In [ ]:
register_image_pair(0, 
                    "../image_stacks/Mistgcamp-3_0003.tif", 
                    "../image_stacks/Mistgcamp-3_0003.tif", 
                    "../image_stacks/")

In [ ]:
#The transforms module uses a (x, y) coordinate convention to be consistent
#with most of the warping literature out there.  But the rest of
#scikit-image uses  a (row, column) convention.

#The following code works for me:

from skimage import io, transform
import numpy as np

image = io.imread('image.jpg')
h, w = image.shape[:2]

rng = np.random.RandomState(0)
xs = rng.randint(0, w - 1, 76)
ys = rng.randint(0, h - 1, 76)

src_pts = np.column_stack([xs, ys])
dst_pts = src_pts

tform = transform.PiecewiseAffineTransform()
tform.estimate(src_pts, dst_pts)

out = transform.warp(image, tform)

import matplotlib.pyplot as plt
plt.imshow(out)
plt.show()

